<a href="https://colab.research.google.com/github/alexmoed/MasterClass/blob/3D-SpatialLM/Export_Classified_pointcloud_SpatialLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Install dependencies if not already installed
!pip install plyfile plotly open3d
import numpy as np
import open3d as o3d
from plyfile import PlyData, PlyElement
import colorsys
import csv
import os
import math
import numpy as np
import plotly.graph_objects as go
from plyfile import PlyData
import os




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 137.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
 

##Splial LM predictions to Segmenation.
This Script is where the each bounding box is used to segment out the points and add a class attribute to those points. This was a added script that I devoloped It does not come with spatial lm

The concept and excution was designed and written by me but I did get help and advice and correction from claude ai

In [3]:
#Essentially this following script imports in the point cloud and applies the bounding boxes and then uses O3d function to seperate out the points contained within the boxes
%cd /content/drive/MyDrive/SpatialLM

#Provide the name of your script and link to the ply file and predictions
SCENE = "KitchenDiner"
INPUT_PLY = f"pcd/{SCENE}_cleaned_v009_rotationfipped.ply"
LAYOUT_FILE = f"{SCENE}_cleaned_v009_rotationfipped.txt" #this is the predictions from spatialLM
OUTPUT_DIR = "pcd"
SCALE_FACTOR = 1.2  #This allows you to scale all the bounding boxes up sometimes the bounding boxes are too small

#Loading pointcloud
def load_point_cloud(ply_path):
    """Load point cloud from PLY file and extract coordinates"""
    print(f"Loading point cloud from {ply_path}")
    plydata = PlyData.read(ply_path)
    vertex = plydata['vertex']
    points = np.vstack([vertex['x'], vertex['y'], vertex['z']]).T
    print(f"Loaded {len(points)} points")
    return plydata, points

def parse_layout_file(path):
    print(f"Parsing layout file: {path}")
    boxes = []
    labels = []
    box_ids = []

    with open(path, 'r') as f:
        for line in f:
            if line.startswith("bbox_"):
                box_id = line.split('=')[0].strip()
                data = line.strip().split("Bbox(")[-1].rstrip(")\n").split(",")

                label = data[0]
                x, y, z = map(float, data[1:4])
                rot = float(data[4])
                dx, dy, dz = map(float, data[5:8])

                boxes.append([x, y, z, dx, dy, dz, rot])
                labels.append(label)
                box_ids.append(box_id)
                print(f"Added box: {box_id} ({label}) at position ({x}, {y}, {z})")

    print(f"Found {len(boxes)} bounding boxes")
    return boxes, labels, box_ids

def classify_points(points, bounding_boxes, SCALE_FACTOR):
    """Classify points using Open3D bounding boxes with optional scaling"""
    print(f"Classifying points with scale factor: {scale_factor}...")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    classification = np.full(len(points), -1, dtype=int)

    for box_idx, box in enumerate(bounding_boxes):
        center = box[0:3]
        dimensions = np.array(box[3:6])
        rotation = box[6]

        #Scaling up the bounding box if not 1
        if scale_factor != 1.0:
            dimensions = dimensions * scale_factor
            print(f"Box {box_idx}: Scaled dimensions from {box[3:6]} to {dimensions}")

        #Using Open 3d to get rotation and orientation of bounding boxes in 3d
        R = o3d.geometry.get_rotation_matrix_from_xyz([0, 0, rotation])
        bbox = o3d.geometry.OrientedBoundingBox(
            center=center,
            R=R,
            extent=dimensions
        )

        #Find points inside this box then classifiying them
        indices = bbox.get_point_indices_within_bounding_box(pcd.points)
        print(f"Box {box_idx}: Found {len(indices)} points")

        #Assign box index to unclassified points. Its -1 for all the points that dont fall under the bounding box
        for idx in indices:
            if classification[idx] == -1:
                classification[idx] = box_idx

    print("Classification complete")
    return classification

def export_classified_ply(plydata, classification, output_dir, scene_name):
    """Export the classified point cloud"""
    #Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    output_file = f"{output_dir}/{scene_name}_classified.ply"

    #Get vertex data
    vertex_elements = plydata['vertex'].data

    #Create new data type with class_id field
    dtype = vertex_elements.dtype.descr + [('class_id', 'i4')]

    #Create new array with expanded type
    new_vertex = np.empty(len(vertex_elements), dtype=dtype)

    #Copy existing data
    for name in vertex_elements.dtype.names:
        new_vertex[name] = vertex_elements[name]

    #Adding the classification data
    new_vertex['class_id'] = classification

    #Saving out the PLY data
    vertex_element = PlyElement.describe(new_vertex, 'vertex')
    PlyData([vertex_element], text=True).write(output_file)

    print(f"Exported classified PLY: {output_file}")

#Declare and print stats
def print_statistics(classification, labels):
    print("\nPoint Classification Statistics:")
    total_points = len(classification)
    classified_points = np.sum(classification >= 0)
    unclassified_points = np.sum(classification == -1)

    print(f"Total points: {total_points}")
    print(f"Classified points: {classified_points} ({classified_points/total_points*100:.1f}%)")
    print(f"Unclassified points: {unclassified_points} ({unclassified_points/total_points*100:.1f}%)")

    for box_idx, label in enumerate(labels):
        point_count = np.sum(classification == box_idx)
        print(f"Box {box_idx} ({label}): {point_count} points ({point_count/total_points*100:.2f}%)")

    unclassified = np.sum(classification == -1)
    print(f"Unclassified: {unclassified} points ({unclassified/total_points*100:.2f}%)")


print("Starting point cloud processing")

#Call functions
plydata, points = load_point_cloud(INPUT_PLY)
bounding_boxes, labels, box_ids = parse_layout_file(LAYOUT_FILE)


classification = classify_points(points, bounding_boxes, SCALE_FACTOR)

export_classified_ply(plydata, classification, OUTPUT_DIR, SCENE)

print_statistics(classification, labels)

print(f"Processing complete with bounding box scale factor: {SCALE_FACTOR}")

/content/drive/MyDrive/SpatialLM
Starting point cloud processing pipeline...
Loading point cloud from pcd/KitchenDiner_cleaned_v009_rotationfipped.ply...
Loaded 1290529 points
Parsing layout file: KitchenDiner_cleaned_v009_rotationfipped.txt
Added box: bbox_0 (tv) at position (5.218529605865479, 5.439421319961548, -1.366396188735962)
Added box: bbox_1 (tv_cabinet) at position (5.118529605865478, 5.489421319961548, -1.916396188735962)
Added box: bbox_2 (combination_sofa) at position (3.5685296058654785, 5.439421319961548, -1.866396188735962)
Added box: bbox_3 (dining_table_combination) at position (3.3185296058654785, 5.439421319961548, -2.116396188735962)
Added box: bbox_4 (leisure_table_and_chair_combination) at position (4.5685296058654785, 3.539421319961548, -2.116396188735962)
Added box: bbox_5 (shoe_cabinet) at position (5.118529605865478, 2.3394213199615477, -1.416396188735962)
Added box: bbox_6 (sideboard) at position (1.9685296058654784, 3.6894213199615478, -1.716396188735962)


The following is just for testing purposes. It rebrings in the ply file we just exported and seperates out one class by class ID.

In [ ]:


#Set the class ID you want to visualize
class_id = 6  # Change this to the class number you want to view (check output above) BBOX number is class ID.

#Declare the scene name
scene = "KitchenDiner"

#Path to the pre-extracted class file (created by the first block of code)
class_file = f"pcd/{scene}_class{class_id}_{labels[class_id]}.ply"

#Warning if file doesnt exist
if not os.path.exists(class_file):
    print(f"Error: Class file {class_file} not found.")
    exit()

#Re-Load the point cloud
pcd = o3d.io.read_point_cloud(class_file)
points = np.asarray(pcd.points)


marker_options = dict(size=2.0, color='rgb(255, 0, 0)', opacity=0.9) #Change color here

#Create and display visualization
fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=marker_options,
    name=f'Class {class_id}'
)])

#Update layout
fig.update_layout(
    scene=dict(aspectmode='data'),
    title=f"Class {class_id} Points from {scene}",
    margin=dict(l=0, r=0, b=0, t=30)
)

fig.show()
print(f"Displaying {len(points)} points for class ID {class_id}")

Displaying 56728 points for class ID 6
